In [3]:
import os
import mne
from mne_connectivity.spectral import spectral_connectivity_epochs
import numpy as np
import pandas as pd
from itertools import combinations

# 使用PLI计算功能连接性
def calculate_connectivity(epochs, fmin, fmax):
    con = spectral_connectivity_epochs(
        epochs, method='pli', mode='multitaper', sfreq=epochs.info['sfreq'],
        fmin=fmin, fmax=fmax, faverage=True, verbose=False, indices=None
    )
    con_data = con.get_data()
    print(f"Connectivity data shape (before reshaping): {con_data.shape}")

    n_channels = len(epochs.info['ch_names'])
    if con_data.shape[0] != n_channels * (n_channels - 1) // 2:
        print(f"Warning: Adjusting data length to match expected channel pairs.")
    con_data = reshape_to_full_matrix(con_data, n_channels)

    print(f"Connectivity data shape (after reshaping): {con_data.shape}")
    return con_data


# 将得到的连接性数据转换为矩阵
def reshape_to_full_matrix(data, n_channels):
    full_matrix = np.zeros((n_channels, n_channels))
    triu_indices = np.array(list(combinations(range(n_channels), 2)))

    if data.ndim == 2:
        data = data[:, 0]

    if len(data) > len(triu_indices):
        print(f"Warning: Data length ({len(data)}) exceeds expected pairs ({len(triu_indices)}). Trimming data.")
        data = data[:len(triu_indices)]
    elif len(data) < len(triu_indices):
        raise ValueError(
            f"Mismatch between data length ({len(data)}) and expected pairs ({len(triu_indices)})."
        )

    # 填充矩阵
    for i, (row, col) in enumerate(triu_indices):
        full_matrix[row, col] = data[i]
        full_matrix[col, row] = data[i]

    return full_matrix


# 处理EEG数据
def process_group_data(folders, event_id, frequency_bands):
    group_connectivity = {band: [] for band in frequency_bands}
    ch_names = None
    file_paths = []

    # 从文件夹中获取所有 EDF 文件路径
    for folder in folders:
        for file in os.listdir(folder):
            if file.endswith('.edf'):
                file_paths.append(os.path.join(folder, file))

    print(f"共发现 {len(file_paths)} 个 EDF 文件。")
    print("开始循环读取 EDF 文件...")
    for file_path in file_paths:
        print(f"文件路径: {file_path}")

        raw = mne.io.read_raw_edf(file_path, preload=True)
        raw.pick_types(eeg=True)

        raw.set_eeg_reference('average', projection=False)
        raw.filter(0.2, None)

        # 提取事件
        events, event_id_dict = mne.events_from_annotations(raw)
        selected_events = events[(events[:, 2] == event_id)]

        time_windows = [(t, t + 0.2) for t in np.arange(0, 1.0, 0.2)]
        connectivity_per_window = {band: [] for band in frequency_bands}

        for tmin, tmax in time_windows:
            epochs = mne.Epochs(raw, selected_events, tmin=tmin, tmax=tmax, preload=True, baseline=None)

            if ch_names is None:
                ch_names = raw.info['ch_names']

            # 计算每个频段的功能连接性
            for band, (fmin, fmax) in frequency_bands.items():
                con = calculate_connectivity(epochs, fmin, fmax)
                connectivity_per_window[band].append(con)

        for band in frequency_bands:
            group_connectivity[band].append(connectivity_per_window[band])

    avg_connectivity = {}
    for band in frequency_bands:
        avg_connectivity[band] = np.mean(group_connectivity[band], axis=0)
    return avg_connectivity, ch_names


# 定义 ASD 与 TD 的文件夹路径
ASD_folders = [
    r'/home/chester/data/connectivity_dataset/ASD/R1/R1-启慧',
    r'/home/chester/data/connectivity_dataset/ASD/R1/R1-咸宁'
]
TD_folders = [r'/home/chester/data/connectivity_dataset/TD/R1/R1-处理完成v1']

# 定义频段
frequency_bands = {'Alpha': (8, 12), 'Theta': (4, 8)}

# 处理每个事件的数据
final_results = {}
for event_id in [1, 2]:
    print(f"Processing Event {event_id}...")
    asd_connectivity, ch_names = process_group_data(ASD_folders, event_id, frequency_bands)
    td_connectivity, _ = process_group_data(TD_folders, event_id, frequency_bands)

    for band in frequency_bands:
        for i, t_start in enumerate(np.arange(0, 1.0, 0.2)):
            asd_matrix = asd_connectivity[band][i]
            td_matrix = td_connectivity[band][i]

            # 保存 ASD 和 TD 的连接性矩阵为 CSV 文件
            asd_df = pd.DataFrame(asd_matrix, index=ch_names, columns=ch_names)
            asd_df.to_csv(f"./Event_{event_id}_{band}_window_{i}_ASD_connectivity.csv")

            td_df = pd.DataFrame(td_matrix, index=ch_names, columns=ch_names)
            td_df.to_csv(f"./Event_{event_id}_{band}_window_{i}_TD_connectivity.csv")

print("计算完成，数据已保存")

Processing Event 1...
共发现 43 个 EDF 文件。
开始循环读取 EDF 文件...
文件路径: /home/chester/data/connectivity_dataset/ASD/R1/R1-启慧/RANK1.edf
Extracting EDF parameters from /home/chester/data/connectivity_dataset/ASD/R1/R1-启慧/RANK1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 22499  =      0.000 ...    74.997 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Filter length: 4951 samples (16.503 s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.250 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.25

Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Not setting metadata
19 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 19 events and 61 original time points ...
0 bad epochs dropped
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
文件路径: /home/chester/data/connectivity_dataset/ASD/R1/R1-启慧/008.edf
Extracting EDF parameters from /home/chester/data/connectivity_dataset/ASD/R1/R1-启慧/008.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 146399  =      0.000 ...   487.997 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...)

/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.250 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.250 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


Used Annotations descriptions: ['1', '2', '3']
Not setting metadata
29 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 29 events and 61 original time points ...
0 bad epochs dropped
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Not setting metadata
29 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 29 events and 61 original time points ...
0 bad epochs dropped
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Not setting metadata
29 matching events found
No baseline correction applied
0 projection items activated
Using data from prelo

...

Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
文件路径: /home/chester/data/connectivity_dataset/TD/R1/R1-处理完成v1/241107-T4-R1.edf
Extracting EDF parameters from /home/chester/data/connectivity_dataset/TD/R1/R1-处理完成v1/241107-T4-R1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 55499  =      0.000 ...   184.997 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband rippl

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.250 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.25

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Filter length: 4951 samples (16.503 s)

Used Annotations descriptions: ['1', '2', '3']
Not setting metadata
9 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 9 events and 61 original time points ...
0 bad epochs dropped
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connect

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.250 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=8.000 Hz corresponds to 1.627 < 5 cycles based on the epoch length 0.203 sec, need at least 0.625 sec epochs or fmin=24.590. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(
/tmp/ipykernel_13417/2849227240.py:10: RuntimeWarning: fmin=4.000 Hz corresponds to 0.813 < 5 cycles based on the epoch length 0.203 sec, need at least 1.25

Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
Connectivity data shape (before reshaping): (441, 1)
Connectivity data shape (after reshaping): (21, 21)
计算完成，数据已保存
